# 0. Imports & define

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# define the path to the data folder
ResultsFolder = r'C:\Users\z5183876\OneDrive - UNSW\Documents\GitHub\24_09_24_Solar_Edge\Results\v_from_i_combined\25_05_01_Results'

# define the path to the site summary file
site_summary_path = r"C:\Users\z5183876\OneDrive - UNSW\Documents\GitHub\24_09_24_Solar_Edge\Data\25_05_01_Newsites_summary.xlsx"

# define the plot template
axis_label_size = 20
axis_num_size = 20
text_size = 20
title_size = 22

# define the figure size for single plot
figure_size = (6, 6)
subplot_size_1_2 = (6, 6)
long_hoz_figsize = (12, 6)
two_by_two_figsize = (12, 12)

# 1. Read the results

## 1.1. Read the results from folders

In [ ]:
# Initialize an empty DataFrame
results_df = pd.DataFrame(
    columns=[
        'Site ID', 'Season', 'File',
        'Mean Frequency (Hz)', 'Mean Period (h)', 'Mismatch Loss (%)'
    ]
)

for folder in os.listdir(ResultsFolder):
    folder_path = os.path.join(ResultsFolder, folder)
    if not os.path.isdir(folder_path):
        continue

    site_id, season = folder.split('_')[:2]

    for file in os.listdir(folder_path):
        if not (file.endswith('.csv') or file.endswith('.xlsx')) or 'combined' not in file:
            continue

        # --- load data ---
        path = os.path.join(folder_path, file)
        df = pd.read_csv(path) if file.endswith('.csv') else pd.read_excel(path)

        # --- compute centroid frequency & period in hours ---
        timestamps = pd.to_datetime(df['Timestamp'])
        dt = (timestamps.iloc[1] - timestamps.iloc[0]).total_seconds()
        y  = df['Sum of I*V (W)'].values
        N  = len(y)

        y_centered = y - np.mean(y)
        y_fft      = np.fft.fft(y_centered)
        freqs      = np.fft.fftfreq(N, d=dt)

        # Only non-negative frequencies
        mask    = freqs >= 0
        freqs_p = freqs[mask]
        amp_p   = np.abs(y_fft[mask])
        power   = amp_p**2

        # centroid freq & period
        f_centroid   = np.sum(freqs_p * power) / np.sum(power)
        T_centroid_s = 1.0 / f_centroid if f_centroid > 0 else np.inf
        T_centroid_h = T_centroid_s / 3600.0 * 2

        # --- plot FFT amplitude spectrum ---
        # plt.figure(figsize=(10,5))
        # plt.plot(freqs_p, amp_p, lw=1)
        # plt.xlim(0, freqs_p.max())
        # plt.yscale('log')
        # plt.xlabel('Frequency (Hz)')
        # plt.ylabel('Amplitude')
        # plt.title(f'FFT of Sum of I*V — Site {site_id}, {season}')
        # plt.grid(True, which='both', ls='--', alpha=0.4)
        # plt.show()

        # print the results
        print(f"Site ID: {site_id}, Season: {season}, File: {file}")
        print(f"Mean Period (h): {T_centroid_h:.3f}")

        # calculate the mismatch loss by taking the percentage diff between the sum of I*V and Pmax
        E_series = np.sum(df['Pmppt (W)'].values)
        E_mpp = np.sum(df['Sum of I*V (W)'].values)
        E_mismatch = E_mpp - E_series
        E_mismatch_loss = E_mismatch / E_mpp * 100.
        print(f"Mismatch Loss (%): {E_mismatch_loss:.3f}")

        # --- append a new row to the DataFrame ---
        results_df.loc[len(results_df)] = [
            site_id, season, file,
            f_centroid, T_centroid_h, E_mismatch_loss
        ]

# --- identify the row with the largest mean period (h) ---
if not results_df.empty:
    idx = results_df['Mean Period (h)'].idxmax()
    max_row = results_df.loc[idx]
    print(">>> Largest mean period:")
    print(max_row.to_frame().T)


if not results_df.empty:
    idx_mismatch = results_df['Mismatch Loss (%)'].idxmax()
    max_mismatch_row = results_df.loc[idx_mismatch]
    print(">>> Largest mismatch loss:")
    print(max_mismatch_row.to_frame().T)

# export the results to an Excel file in the Results folder
results_df.to_excel(os.path.join(ResultsFolder, 'combined_results.xlsx'), index=False)

## 1.2. Add the summary information to the combined df

In [ ]:
# 1.2. Add the summary information to the combined df

# Read the site summary Excel file
site_summary_df = pd.read_excel(site_summary_path, sheet_name='Sheet1')

# Ensure 'Site ID' columns are the same type (string is safest)
results_df['Site ID'] = results_df['Site ID'].astype(str)
site_summary_df['Site ID'] = site_summary_df['Site ID'].astype(str)

# Merge the results DataFrame with the site summary DataFrame on 'Site ID'
combined_df = pd.merge(results_df, site_summary_df, on='Site ID', how='left')

# remove the line with mismatch loss >= 100%
combined_df = combined_df[combined_df['Mismatch Loss (%)'] < 100]

# Export the merged DataFrame to an Excel file in the Results folder
combined_df.to_excel(os.path.join(ResultsFolder, 'combined_results_with_summary.xlsx'), index=False)

# 2. Visualise the mismatch losses

### 2.2.1. Overall hist

In [ ]:
# plot a histogram of the mismatch loss
plt.figure(figsize=figure_size)
plt.hist(combined_df['Mismatch Loss (%)'], bins=9, color='blue', alpha=0.7, edgecolor='black')

# Calculate mean, median, and std
mean_val = combined_df['Mismatch Loss (%)'].mean()
median_val = combined_df['Mismatch Loss (%)'].median()
std_val = combined_df['Mismatch Loss (%)'].std()

# Plot vertical lines for mean and median
plt.axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f"Mean = {mean_val:.2f}%")
plt.axvline(median_val, color='black', linestyle='-.', linewidth=2, label=f"Median = {median_val:.2f}%")

plt.xlabel('Mismatch Loss (%)', fontsize=axis_label_size)
plt.ylabel('Frequency', fontsize=axis_label_size)
plt.title(f'Histogram of Mismatch Loss\nMean = {mean_val:.2f}% ± {std_val:.2f}%', fontsize=title_size)
plt.grid(axis='y', alpha=0.75)
plt.xticks(fontsize=axis_num_size)
plt.yticks(fontsize=axis_num_size)
plt.legend(fontsize=axis_num_size)
plt.show()

### 2.2.2. Plot against mean frequency

In [ ]:
# plot the mismatch loss vs mean period, colored by orientation
plt.figure(figsize=figure_size)
orientations = combined_df['Orientation'].unique()
colors = plt.cm.tab10.colors  # up to 10 unique colors

for idx, orientation in enumerate(orientations):
    subset = combined_df[combined_df['Orientation'] == orientation]
    plt.scatter(
        subset['Mean Period (h)'],
        subset['Mismatch Loss (%)'],
        color=colors[idx % len(colors)],
        alpha=0.7,
        label=str(orientation)
    )

plt.xlabel('Mean Period (h)', fontsize=axis_label_size)
plt.ylabel('Mismatch Loss (%)', fontsize=axis_label_size)
plt.title('Mismatch Loss vs Mean Period\n(colored by Orientation)', fontsize=title_size)
plt.grid(alpha=0.75)
plt.xticks(fontsize=axis_num_size)
plt.yticks(fontsize=axis_num_size)
plt.legend(title='Orientation', fontsize=axis_num_size)
plt.show()



# plot the mismatch loss vs mean period, colored by Shade
plt.figure(figsize=figure_size)
shades = combined_df['Shade'].unique()
colors = plt.cm.tab10.colors  # up to 10 unique colors

for idx, shade in enumerate(shades):
    subset = combined_df[combined_df['Shade'] == shade]
    plt.scatter(
        subset['Mean Period (h)'],
        subset['Mismatch Loss (%)'],
        color=colors[idx % len(colors)],
        alpha=0.7,
        label=str(shade)
    )

plt.xlabel('Mean Period (h)', fontsize=axis_label_size)
plt.ylabel('Mismatch Loss (%)', fontsize=axis_label_size)
plt.title('Mismatch Loss vs Mean Period\n(colored by Shade)', fontsize=title_size)
plt.grid(alpha=0.75)
plt.xticks(fontsize=axis_num_size)
plt.yticks(fontsize=axis_num_size)
plt.legend(title='Shade', fontsize=axis_num_size)
plt.show()


# 2-column, 1-row subplot: left for 'Multi', right for 'Single', with straight line fit and slope

fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)

# Filter for 'Single' and 'Multi' (assuming a column 'Type' exists; adjust if needed)
multi_df = combined_df[combined_df['Orientation'].str.lower() == 'multi']
single_df = combined_df[combined_df['Orientation'].str.lower() == 'single']
# Left: Multi
orientations = multi_df['Orientation'].unique()
colors = plt.cm.tab10.colors
ax = axes[0]
for idx, orientation in enumerate(orientations):
    subset = multi_df[multi_df['Orientation'] == orientation]
    ax.scatter(
        subset['Mean Period (h)'],
        subset['Mismatch Loss (%)'],
        color='black',
    )
# Fit line
if not multi_df.empty:
    x = multi_df['Mean Period (h)'].values
    y = multi_df['Mismatch Loss (%)'].values
    if len(x) > 1:
        coeffs = np.polyfit(x, y, 1)
        slope = coeffs[0]
        xfit = np.linspace(np.nanmin(x), np.nanmax(x), 100)
        yfit = np.polyval(coeffs, xfit)
        ax.plot(xfit, yfit, 'k--', label=f'Fit: slope={slope:.2f}')
        print(f"Multi: Slope = {slope:.4f}")
    else:
        slope = np.nan
ax.set_xlabel('Mean Period (h)', fontsize=axis_label_size)
ax.set_ylabel('Mismatch Loss (%)', fontsize=axis_label_size)
ax.set_title('Multi', fontsize=title_size)
ax.tick_params(axis='y', labelsize=axis_num_size)
ax.tick_params(axis='x', labelsize=axis_num_size)
ax.grid(alpha=0.75)
ax.legend(fontsize=axis_num_size, loc='lower left')

# Right: Single
orientations = single_df['Orientation'].unique()
ax = axes[1]
for idx, orientation in enumerate(orientations):
    subset = single_df[single_df['Orientation'] == orientation]
    ax.scatter(
        subset['Mean Period (h)'],
        subset['Mismatch Loss (%)'],
        color='black',
    )
# Fit line
if not single_df.empty:
    x = single_df['Mean Period (h)'].values
    y = single_df['Mismatch Loss (%)'].values
    if len(x) > 1:
        coeffs = np.polyfit(x, y, 1)
        slope = coeffs[0]
        xfit = np.linspace(np.nanmin(x), np.nanmax(x), 100)
        yfit = np.polyval(coeffs, xfit)
        ax.plot(xfit, yfit, 'k--', label=f'Fit: slope={slope:.2f}')
        print(f"Single: Slope = {slope:.4f}")
    else:
        slope = np.nan
ax.set_xlabel('Mean Period (h)', fontsize=axis_label_size)
ax.tick_params(axis='y', labelsize=axis_num_size)
ax.tick_params(axis='x', labelsize=axis_num_size)
ax.set_title('Single', fontsize=title_size)
ax.grid(alpha=0.75)
ax.legend(fontsize=axis_num_size, loc='lower left')

plt.suptitle('Mismatch Loss vs Mean Period', fontsize=title_size)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()




# 2-column, 1-row subplot: left for 'Shaded', right for 'Unshaded', with straight line fit and slope

fig, axes = plt.subplots(1, 2, figsize=(12, 6), sharey=True)

# Filter for 'Shaded' (yes) and 'Unshaded' (no)
shaded_df = combined_df[combined_df['Shade'].str.lower() == 'yes']
unshaded_df = combined_df[combined_df['Shade'].str.lower() == 'no']

# Left: Shaded
shades = shaded_df['Shade'].unique()
colors = plt.cm.tab10.colors
ax = axes[0]
for idx, shade in enumerate(shades):
    subset = shaded_df[shaded_df['Shade'] == shade]
    ax.scatter(
        subset['Mean Period (h)'],
        subset['Mismatch Loss (%)'],
        color='black',
    )
# Fit line
if not shaded_df.empty:
    x = shaded_df['Mean Period (h)'].values
    y = shaded_df['Mismatch Loss (%)'].values
    if len(x) > 1:
        coeffs = np.polyfit(x, y, 1)
        slope = coeffs[0]
        xfit = np.linspace(np.nanmin(x), np.nanmax(x), 100)
        yfit = np.polyval(coeffs, xfit)
        ax.plot(xfit, yfit, 'k--', label=f'Fit: slope={slope:.2f}')
        print(f"Shaded: Slope = {slope:.4f}")
    else:
        slope = np.nan
ax.set_xlabel('Mean Period (h)', fontsize=axis_label_size)
ax.set_ylabel('Mismatch Loss (%)', fontsize=axis_label_size)
ax.set_title('Shaded', fontsize=title_size)
ax.tick_params(axis='y', labelsize=axis_num_size)
ax.tick_params(axis='x', labelsize=axis_num_size)
ax.grid(alpha=0.75)
ax.legend(fontsize=axis_num_size, loc='lower left')

# Right: Unshaded
shades = unshaded_df['Shade'].unique()
ax = axes[1]
for idx, shade in enumerate(shades):
    subset = unshaded_df[unshaded_df['Shade'] == shade]
    ax.scatter(
        subset['Mean Period (h)'],
        subset['Mismatch Loss (%)'],
        color='black',
    )
# Fit line
if not unshaded_df.empty:
    x = unshaded_df['Mean Period (h)'].values
    y = unshaded_df['Mismatch Loss (%)'].values
    if len(x) > 1:
        coeffs = np.polyfit(x, y, 1)
        slope = coeffs[0]
        xfit = np.linspace(np.nanmin(x), np.nanmax(x), 100)
        yfit = np.polyval(coeffs, xfit)
        ax.plot(xfit, yfit, 'k--', label=f'Fit: slope={slope:.2f}')
        print(f"Unshaded: Slope = {slope:.4f}")
    else:
        slope = np.nan
ax.set_xlabel('Mean Period (h)', fontsize=axis_label_size)
ax.tick_params(axis='y', labelsize=axis_num_size)
ax.tick_params(axis='x', labelsize=axis_num_size)
ax.set_title('Unshaded', fontsize=title_size)
ax.grid(alpha=0.75)
ax.legend(fontsize=axis_num_size, loc='lower left')

plt.suptitle('Mismatch Loss vs Mean Period (by Shading)', fontsize=title_size)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

### 2.2.3. Group by country hist

In [ ]:

# Group by country and calculate mean and std for mismatch loss
country_stats = combined_df.groupby('Country')['Mismatch Loss (%)'].agg(['mean', 'std'])

plt.figure(figsize=figure_size)
plt.bar(
    country_stats.index,
    country_stats['mean'],
    yerr=country_stats['std'],
    color='blue',
    alpha=0.7,
    capsize=5
)
plt.xlabel('Country', fontsize=axis_label_size)
plt.ylabel('Mismatch Loss (%)', fontsize=axis_label_size)
plt.title('Mismatch Loss by Country', fontsize=title_size)
plt.grid(axis='y', alpha=0.75)
plt.xticks(rotation=45, fontsize=axis_num_size)
plt.yticks(fontsize=axis_num_size)
plt.tight_layout()
plt.show()

### 2.2.4. Group by orientation hist

In [ ]:
# group by the orientation and calculate mean and std for mismatch loss
orientation_stats = combined_df.groupby('Orientation')['Mismatch Loss (%)'].agg(['mean', 'std'])

plt.figure(figsize=figure_size)
bars = plt.bar(
    orientation_stats.index,
    orientation_stats['mean'],
    yerr=orientation_stats['std'],
    color='blue',
    alpha=0.7,
    capsize=5
)
plt.xlabel('Orientation', fontsize=axis_label_size)
plt.ylabel('Mismatch Loss (%)', fontsize=axis_label_size)
plt.title('Mismatch Loss by Orientation', fontsize=title_size)
plt.grid(axis='y', alpha=0.75)
plt.xticks(rotation=45, fontsize=axis_num_size)
plt.yticks(fontsize=axis_num_size)
plt.tight_layout()
plt.ylim(0, 30)
# Add value and std on top of each bar
for bar, mean, std in zip(bars, orientation_stats['mean'], orientation_stats['std']):
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + std + 0.2,
        f"{mean:.2f}±{std:.2f}",
        ha='center',
        va='bottom',
        fontsize=axis_num_size
    )
plt.show()

### 2.2.5. Group by shading

In [ ]:
# group by shading and calculate mean and std for mismatch loss
shading_stats = combined_df.groupby('Shade')['Mismatch Loss (%)'].agg(['mean', 'std'])
plt.figure(figsize=figure_size)
bars = plt.bar(
    shading_stats.index,
    shading_stats['mean'],
    yerr=shading_stats['std'],
    color='blue',
    alpha=0.7,
    capsize=5
)
plt.xlabel('Shading', fontsize=axis_label_size)
plt.ylabel('Mismatch Loss (%)', fontsize=axis_label_size)
plt.title('Mismatch Loss by Shading', fontsize=title_size)
plt.grid(axis='y', alpha=0.75)
plt.xticks(rotation=45, fontsize=axis_num_size)
plt.yticks(fontsize=axis_num_size)
plt.tight_layout()
plt.ylim(0, 30)
# Add value and std on top of each bar
for bar, mean, std in zip(bars, shading_stats['mean'], shading_stats['std']):
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + std + 0.2,
        f"{mean:.2f}±{std:.2f}",
        ha='center',
        va='bottom',
        fontsize=axis_num_size
    )
plt.show()

### 2.2.6. Group by season

In [ ]:
# group by season and calculate mean and std for mismatch loss
season_stats = combined_df.groupby('Season')['Mismatch Loss (%)'].agg(['mean', 'std'])
plt.figure(figsize=figure_size)
bars = plt.bar(
    season_stats.index,
    season_stats['mean'],
    yerr=season_stats['std'],
    color='blue',
    alpha=0.7,
    capsize=5
)
plt.xlabel('Season', fontsize=axis_label_size)
plt.ylabel('Mismatch Loss (%)', fontsize=axis_label_size)
plt.title('Mismatch Loss by Season', fontsize=title_size)
plt.grid(axis='y', alpha=0.75)
plt.xticks(rotation=45, fontsize=axis_num_size)
plt.yticks(fontsize=axis_num_size)
plt.tight_layout()
plt.ylim(0, 30)
# Add value and std on top of each bar
for bar, mean, std in zip(bars, season_stats['mean'], season_stats['std']):
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + std + 0.2,
        f"{mean:.2f}±{std:.2f}",
        ha='center',
        va='bottom',
        fontsize=axis_num_size-10
    )
plt.show()

### 2.2.7. Plot against sytem size

In [ ]:
# only plot the site with multiple orientations
multiple_orientations = combined_df[combined_df['Orientation'].str.lower() == 'multi']

# create a new figure with 2x2 subplots
fig, axes = plt.subplots(2, 2, figsize=two_by_two_figsize, sharey=True)
fig.subplots_adjust(hspace=0.4, wspace=0.4)

# group by season and plot the mismatch loss vs system size
seasons = multiple_orientations['Season'].unique()
for idx, season in enumerate(seasons):
    ax = axes.flatten()[idx]  # Use a different subplot for each season
    subset = multiple_orientations[multiple_orientations['Season'] == season]
    ax.scatter(
        subset['kWp'],
        subset['Mismatch Loss (%)'],
        color='black'
    )
    # Fit line
    if not subset.empty:
        x = subset['kWp'].values
        y = subset['Mismatch Loss (%)'].values
        if len(x) > 1:
            coeffs = np.polyfit(x, y, 1)
            slope = coeffs[0]
            xfit = np.linspace(np.nanmin(x), np.nanmax(x), 100)
            yfit = np.polyval(coeffs, xfit)
            ax.plot(xfit, yfit, 'k--', label=f'Fit: slope={slope:.2f}')
            print(f"{season}: Slope = {slope:.4f}")
        else:
            slope = np.nan
    ax.set_xlabel('System Size (kW)', fontsize=axis_label_size)
    ax.set_ylabel('Mismatch Loss (%)', fontsize=axis_label_size)
    ax.set_title(f'{season}', fontsize=title_size)
    ax.tick_params(axis='y', labelsize=axis_num_size)
    ax.tick_params(axis='x', labelsize=axis_num_size)
    ax.grid(alpha=0.75)
    ax.legend(fontsize=axis_num_size, loc='lower left')  # Only fit line in legend

plt.suptitle('Mismatch Loss vs System Size by Season (Multi Orientation)', fontsize=title_size)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

# only plot the site with single orientation
single_orientations = combined_df[combined_df['Orientation'].str.lower() == 'single']

# create a new figure with 2x2 subplots
fig, axes = plt.subplots(2, 2, figsize=two_by_two_figsize, sharey=True)
fig.subplots_adjust(hspace=0.4, wspace=0.4)

# group by season and plot the mismatch loss vs system size
seasons = single_orientations['Season'].unique()
for idx, season in enumerate(seasons):
    ax = axes.flatten()[idx]  # Use a different subplot for each season
    subset = single_orientations[single_orientations['Season'] == season]
    ax.scatter(
        subset['kWp'],
        subset['Mismatch Loss (%)'],
        color='black'
    )
    # Fit line
    if not subset.empty:
        x = subset['kWp'].values
        y = subset['Mismatch Loss (%)'].values
        if len(x) > 1:
            coeffs = np.polyfit(x, y, 1)
            slope = coeffs[0]
            xfit = np.linspace(np.nanmin(x), np.nanmax(x), 100)
            yfit = np.polyval(coeffs, xfit)
            ax.plot(xfit, yfit, 'k--', label=f'Fit: slope={slope:.2f}')
            print(f"{season}: Slope = {slope:.4f}")
        else:
            slope = np.nan
    ax.set_xlabel('System Size (kW)', fontsize=axis_label_size)
    ax.set_ylabel('Mismatch Loss (%)', fontsize=axis_label_size)
    ax.set_title(f'{season}', fontsize=title_size)
    ax.tick_params(axis='y', labelsize=axis_num_size)
    ax.tick_params(axis='x', labelsize=axis_num_size)
    ax.grid(alpha=0.75)
    ax.legend(fontsize=axis_num_size, loc='lower left')  # Only fit line in legend

plt.suptitle('Mismatch Loss vs System Size by Season (Single Orientation)', fontsize=title_size)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

# 3. J0 and bypass investigation

In [ ]:
# define the site id
site_id = '4093266'
# define the season
season = 'spring'
# look for folder in the Results folder that contains both site id and season

# Find the folder
target_folder = None
for folder in os.listdir(ResultsFolder):
    if site_id in folder and season in folder.lower():
        target_folder = os.path.join(ResultsFolder, folder)
        break

if target_folder is not None:
    print(f"Found folder: {target_folder}")
    # List files in the folder
    files = os.listdir(target_folder)
    # look for a csv file that contains the word j0
    for file in files:
        if file.endswith('.csv') and 'j0' in file:
            target_file = os.path.join(target_folder, file)
            print(f"Found target file: {target_file}")
            # read the csv file
            df = pd.read_csv(target_file)
            # plot each column in a plot
            for col in df.columns[1:]:
                plt.figure(figsize=figure_size)
                plt.plot(df['Timestamp'], df[col], label=col)
                plt.xlabel('Timestamp', fontsize=axis_label_size)
                plt.ylabel(col, fontsize=axis_label_size)
                plt.title(f'{col} vs Timestamp', fontsize=title_size)
                # limit x ticks to have steps of 3 hours
                plt.xticks(df['Timestamp'][::12], rotation=90, fontsize=axis_num_size-10)
                plt.yticks(fontsize=axis_num_size)
                plt.grid(alpha=0.75)
                plt.legend(fontsize=axis_num_size)
                plt.yscale('log')
                plt.tight_layout()
                # set the x limit to the first 24*60/5 points
                plt.xlim(0, 12*60/5)
                plt.show()